# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [3]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [4]:
client = MongoClient('mongodb://mongo:27017/')
database = client['exercices']
collection = database['youtube']

In [5]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [6]:
df_youtube = df_youtube.dropna()

### Importer les données

In [7]:
collection.delete_many({})
collection.insert_many(df_youtube.to_dict('records'))

In [8]:
collection.find_one()

{'_id': ObjectId('5fbbc3aeb3fc121f4fad5991'),
 'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09}

## Question 1  

In [9]:
cur = collection.find({"channel_title":"Apple"})
print(len(list(cur)))

11


## Question 2

In [21]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_id", "sum" : {"$sum" : 1}}}])
list(cur)

[{'_id': 25, 'sum': 623},
 {'_id': 1, 'sum': 378},
 {'_id': 2, 'sum': 116},
 {'_id': 22, 'sum': 882},
 {'_id': 20, 'sum': 82},
 {'_id': 27, 'sum': 334},
 {'_id': 43, 'sum': 2},
 {'_id': 24, 'sum': 1601},
 {'_id': 28, 'sum': 512},
 {'_id': 19, 'sum': 48},
 {'_id': 29, 'sum': 14},
 {'_id': 15, 'sum': 116},
 {'_id': 17, 'sum': 410},
 {'_id': 23, 'sum': 755},
 {'_id': 26, 'sum': 869},
 {'_id': 10, 'sum': 1250}]

## Question 3

In [26]:
cur = collection.find({},{"tags":1,"_id":1})
cur = list(cur)
li = [(x["tags"].split('|'),x["_id"]) for x in cur]
for tag in li:
    tags, _id = tag
    collection.update_one( {"_id":_id}, {"$set":{"tags":tags}} )

collection.find_one()

{'_id': ObjectId('5fbbc3aeb3fc121f4fad5991'),
 'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life'],
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09}

## Question 4

In [22]:
cur = collection.find().sort([("views", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('5fbbc3aeb3fc121f4fad6104'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09},
 {'_id': ObjectId('5fbbc3aeb3fc121f4fad602f'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09},
 {'_id': ObjectId('5fbbc3aeb3fc121f4fad5a42'),
  'video_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': 'Dusk Till Dawn|Pop|RCA Records 

## Question 5

In [23]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_id", "averageViewsByCategory" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 10, 'averageViewsByCategory': 1176553.612},
 {'_id': 20, 'averageViewsByCategory': 681081.8170731707},
 {'_id': 25, 'averageViewsByCategory': 540955.5569823436},
 {'_id': 26, 'averageViewsByCategory': 537665.1806674339},
 {'_id': 15, 'averageViewsByCategory': 651404.3879310344},
 {'_id': 17, 'averageViewsByCategory': 728434.3243902439},
 {'_id': 23, 'averageViewsByCategory': 1240073.294039735},
 {'_id': 29, 'averageViewsByCategory': 1110334.2142857143},
 {'_id': 24, 'averageViewsByCategory': 1154868.4397251718},
 {'_id': 28, 'averageViewsByCategory': 924730.548828125},
 {'_id': 19, 'averageViewsByCategory': 464041.0833333333},
 {'_id': 27, 'averageViewsByCategory': 547582.4700598803},
 {'_id': 43, 'averageViewsByCategory': 8492.5},
 {'_id': 1, 'averageViewsByCategory': 1039472.6560846561},
 {'_id': 2, 'averageViewsByCategory': 607693.8706896552},
 {'_id': 22, 'averageViewsByCategory': 971532.7573696146}]

## Question 6 

In [25]:
#cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "averageLikesByChannel" : {"$avg" : "$likes"}}}])
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "number" : {"$avg" : "$likes"}}},{"$sort":{"number":-1}}])
list(cur)

[{'_id': 'ZaynVEVO', 'number': 1431683.0},
 {'_id': 'ibighit', 'number': 1371766.25},
 {'_id': 'melanie martinez', 'number': 911871.5},
 {'_id': 'BETNetworks', 'number': 769687.625},
 {'_id': 'jypentertainment', 'number': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'number': 653443.5384615385},
 {'_id': 'Wengie', 'number': 567235.2},
 {'_id': 'Alesso', 'number': 509417.28571428574},
 {'_id': 'PewDiePie', 'number': 508647.8},
 {'_id': 'shakiraVEVO', 'number': 434302.14285714284},
 {'_id': 'Ed Sheeran', 'number': 395654.1666666667},
 {'_id': 'Dude Perfect', 'number': 389697.1111111111},
 {'_id': 'Conor Maynard', 'number': 389273.0},
 {'_id': 'TaylorSwiftVEVO', 'number': 367455.6666666667},
 {'_id': 'Liza Koshy', 'number': 365146.9583333333},
 {'_id': 'Linkin Park', 'number': 356475.7272727273},
 {'_id': 'jbalvinVEVO', 'number': 345130.4285714286},
 {'_id': 'Primitive Technology', 'number': 340659.25},
 {'_id': 'Gabbie Hanna', 'number': 339488.0},
 {'_id': 'Bhad Bhabie', 'number': 31